In [1]:
import pandas as pd
import requests
import json
from dateutil.relativedelta import relativedelta
from datetime import date

In [2]:
url = 'https://opendart.fss.or.kr/api/list.json'
key = '########################################'
formtype = 'A001'
bgn_de='20150101'
end_de='20151231'

payload = {
    'crtfc_key':key,
    'pblntf_detail_ty': formtype,
    'bgn_de': bgn_de,
    'end_de': end_de,
    'page_count':100 # 페이지 카운트는 최댓값 100
    
}
req = requests.get(url, params=payload).text
search_result = json.loads(req)
search_result

{'status': '100', 'message': 'corp_code가 없는 경우 검색기간은 3개월만 가능합니다.'}

### 이런 오류가 생기므로 3개월로 데이터를 쪼개봅니다.

In [3]:
url = 'https://opendart.fss.or.kr/api/list.json'
key = '########################################'
formtype = 'A001'
bgn_de = date(2015,1,1)
end_de= date(2015,12,31)
search_type = {
    'A001' : '사업보고서'
}

quarter_date = [bgn_de, bgn_de + relativedelta(months=3),bgn_de+relativedelta(months=6), bgn_de+relativedelta(months=9), end_de]

payload = {
    'crtfc_key':key,
    'pblntf_detail_ty': formtype,
    'page_count':100 # 페이지 카운트는 최댓값 100
    
}

total_search_result = []
for idx in range(len(quarter_date)-1):
    print('{0}분기 {1} 데이터 수집 시작'.format(idx+1, search_type[formtype]))
    payload['bgn_de'] = quarter_date[idx].strftime('%Y%m%d')
    payload['end_de'] = quarter_date[idx+1].strftime('%Y%m%d')    
    req1 = requests.get(url, params=payload).text
    if json.loads(req1)['status'] != '000':
        print('조회된 데이터가 없습니다')
        break
    total_page= json.loads(req1)['total_page']    
    print('{0}분기 페이지 개수 : {1}페이지'.format(idx+1, total_page))
    
    for i in range(1, total_page):
        payload['page_no'] = i
        req2 = requests.get(url, params=payload).text
        quarter_page_data = json.loads(req2)
        total_search_result += quarter_page_data['list']
    print('{0}분기 {1} 데이터 수집 종료'.format(idx+1, search_type[formtype]))
        
df = pd.DataFrame(total_search_result)

1분기 사업보고서 데이터 수집 시작
1분기 페이지 개수 : 24페이지
1분기 사업보고서 데이터 수집 종료
2분기 사업보고서 데이터 수집 시작
2분기 페이지 개수 : 9페이지
2분기 사업보고서 데이터 수집 종료
3분기 사업보고서 데이터 수집 시작
3분기 페이지 개수 : 1페이지
3분기 사업보고서 데이터 수집 종료
4분기 사업보고서 데이터 수집 시작
4분기 페이지 개수 : 1페이지
4분기 사업보고서 데이터 수집 종료


In [4]:
df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00136864,신원,009270,Y,[기재정정]사업보고서 (2014.12),20150401002372,신원,20150401,연
1,00110893,대신증권,003540,Y,[기재정정]사업보고서 (2014.12),20150401002266,대신증권,20150401,연
2,00306162,상상인,038540,K,[기재정정]사업보고서 (2014.12),20150401002155,상상인,20150401,연
3,00407814,크리스탈,083790,K,[기재정정]사업보고서 (2014.12),20150401002122,크리스탈,20150401,연
4,00493510,뉴프렉스,085670,K,[기재정정]사업보고서 (2014.12),20150401001891,뉴프렉스,20150401,연
...,...,...,...,...,...,...,...,...,...
3095,00681373,디지캡,197140,K,[기재정정]사업보고서 (2014.12),20150408001672,디지캡,20150408,정
3096,00173740,신한,005450,Y,[기재정정]사업보고서 (2014.12),20150408000787,신한,20150408,연
3097,00164469,현대하이스코,010520,E,[기재정정]사업보고서 (2014.12),20150407002061,현대하이스코,20150407,정연
3098,00874195,코썬바이오,204990,K,사업보고서 (2014.12),20150407001995,코썬바이오,20150407,


In [5]:
df.columns = ['고유번호', '기업명', '종목코드', '법인구분', '보고서명', '접수번호','공시제출인명', '접수일자', '비고']
df

,고유번호,기업명,종목코드,법인구분,보고서명,접수번호,공시제출인명,접수일자,비고
0,00136864,신원,009270,Y,[기재정정]사업보고서 (2014.12),20150401002372,신원,20150401,연
1,00110893,대신증권,003540,Y,[기재정정]사업보고서 (2014.12),20150401002266,대신증권,20150401,연
2,00306162,상상인,038540,K,[기재정정]사업보고서 (2014.12),20150401002155,상상인,20150401,연
3,00407814,크리스탈,083790,K,[기재정정]사업보고서 (2014.12),20150401002122,크리스탈,20150401,연
4,00493510,뉴프렉스,085670,K,[기재정정]사업보고서 (2014.12),20150401001891,뉴프렉스,20150401,연
...,...,...,...,...,...,...,...,...,...
3095,00681373,디지캡,197140,K,[기재정정]사업보고서 (2014.12),20150408001672,디지캡,20150408,정
3096,00173740,신한,005450,Y,[기재정정]사업보고서 (2014.12),20150408000787,신한,20150408,연
3097,00164469,현대하이스코,010520,E,[기재정정]사업보고서 (2014.12),20150407002061,현대하이스코,20150407,정연
3098,00874195,코썬바이오,204990,K,사업보고서 (2014.12),20150407001995,코썬바이오,20150407,


In [6]:
def get_corp_class(str_):
    code = {'Y' : '유가','K' : '코스닥','N' : '코넥스','E' : '기타'}
    return code[str_]

def get_extra_class(str_):
    statements = {
        '유' : '본 공시사항은 한국거래소 유가증권시장본부 소관임',
        '코' : '본 공시사항은 한국거래소 코스닥시장본부 소관임',
        '채' : '본 문서는 한국거래소 채권상장법인 공시사항임',
        '넥' : '본 문서는 한국거래소 코넥스시장 소관임',
        '공' : '본 공시사항은 공정거래위원회 소관임',
        '연' : '본 보고서는 연결부분을 포함한 것임',
        '정' : '본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람',
        '철' : '본 보고서는 철회(간주)되었으니 관련 철회신고서(철회간주안내)를 참고하시기 바람',
    }
    statement = '' 
    for code in str_:
        statement += statements[code]+ ' '        
    return statement

df['법인구분'] = df['법인구분'].apply(lambda x: get_corp_class(x))
df['비고'] = df['비고'].apply(lambda x: get_extra_class(x))
df   

,고유번호,기업명,종목코드,법인구분,보고서명,접수번호,공시제출인명,접수일자,비고
0,00136864,신원,009270,유가,[기재정정]사업보고서 (2014.12),20150401002372,신원,20150401,본 보고서는 연결부분을 포함한 것임
1,00110893,대신증권,003540,유가,[기재정정]사업보고서 (2014.12),20150401002266,대신증권,20150401,본 보고서는 연결부분을 포함한 것임
2,00306162,상상인,038540,코스닥,[기재정정]사업보고서 (2014.12),20150401002155,상상인,20150401,본 보고서는 연결부분을 포함한 것임
3,00407814,크리스탈,083790,코스닥,[기재정정]사업보고서 (2014.12),20150401002122,크리스탈,20150401,본 보고서는 연결부분을 포함한 것임
4,00493510,뉴프렉스,085670,코스닥,[기재정정]사업보고서 (2014.12),20150401001891,뉴프렉스,20150401,본 보고서는 연결부분을 포함한 것임
...,...,...,...,...,...,...,...,...,...
3095,00681373,디지캡,197140,코스닥,[기재정정]사업보고서 (2014.12),20150408001672,디지캡,20150408,본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람
3096,00173740,신한,005450,유가,[기재정정]사업보고서 (2014.12),20150408000787,신한,20150408,본 보고서는 연결부분을 포함한 것임
3097,00164469,현대하이스코,010520,기타,[기재정정]사업보고서 (2014.12),20150407002061,현대하이스코,20150407,본 보고서 제출 후 정정신고가 있으니 관련 보고서를 참조하시기 바람 본 보고서는 연...
3098,00874195,코썬바이오,204990,코스닥,사업보고서 (2014.12),20150407001995,코썬바이오,20150407,


In [ ]:
df.to_pickle('2015_yearly_reports.pickle')